In [1]:
from typing import Any

import torch
from torch import nn
from torch.autograd import Function

import torchvision.transforms as T
from torchvision.models.resnet import resnet50, ResNet50_Weights

import pytorch_lightning as L

from torchmetrics import Accuracy

from wilds import get_dataset
from wilds.common.data_loaders import get_eval_loader, get_train_loader
from wilds.common.grouper import CombinatorialGrouper

from utils import get_backbone_from_ckpt, DomainMapper

In [2]:
class ReverseLayerF(Function):
    @staticmethod
    def forward(ctx, x, alpha):
        ctx.alpha = alpha

        return x.view_as(x)

    @staticmethod
    def backward(ctx, grad_output):
        output = grad_output.neg() * ctx.alpha

        return output, None

In [3]:
class EarlyDANN(nn.Module):
    def __init__(self, weights, alpha=0.15) -> None:
        super().__init__()
        self.backbone = self._make_backbone(weights)
        self.disc_head = nn.Linear(64+256+512+1024+2048, 3)

        self.crit_pred = nn.CrossEntropyLoss()
        self.crit_disc = nn.CrossEntropyLoss()

        self.alpha = alpha
    
    def forward(self, x):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        z0 = x.mean(dim=[2,3])

        x = self.backbone.layer1(x)

        z1 = x.mean(dim=[2,3])

        x = self.backbone.layer2(x)
        
        z2 = x.mean(dim=[2,3])

        x = self.backbone.layer3(x)
        
        z3 = x.mean(dim=[2,3])

        x = self.backbone.layer4(x)

        z4 = x.mean(dim=[2,3])

        x = self.backbone.avgpool(x)

        f = torch.flatten(x, 1)

        z_ = torch.cat([z0, z1, z2, z3, z4], dim=1)
        z_ = ReverseLayerF.apply(z_, self.alpha)

        y = self.backbone.fc(f)
        z = self.disc_head(z_)

        return y, z
    
    def embed(self, x: torch.Tensor):
        x = self.backbone.conv1(x)
        x = self.backbone.bn1(x)
        x = self.backbone.relu(x)
        x = self.backbone.maxpool(x)

        z0 = x.mean(dim=[2,3])

        x = self.backbone.layer1(x)
        z1 = x.mean(dim=[2,3])

        x = self.backbone.layer2(x)
        z2 = x.mean(dim=[2,3])
        
        x = self.backbone.layer3(x)
        z3 = x.mean(dim=[2,3])
        
        x = self.backbone.layer4(x)
        z4 = x.mean(dim=[2,3])

        x = self.backbone.avgpool(x).squeeze()
    
        return {
            "z0": z0,
            "z1": z1,
            "z2": z2,
            "z3": z3,
            "z4": z4,
            "x": x
        }
    
    def _make_backbone(self, weights):
        if weights == "scratch":
            backbone = resnet50(num_classes=2)
        elif weights == "ImageNet":
            backbone = resnet50(weights=ResNet50_Weights.DEFAULT)
            backbone.fc = nn.Linear(2048, 2)
        else:
            backbone = resnet50(num_classes=2)
            missing_keys, unexpected_keys = backbone.load_state_dict(get_backbone_from_ckpt(weights), strict=False)
            print("missing:", missing_keys, "unexpected:", unexpected_keys)

        return backbone

In [4]:
class SimpleCNN(L.LightningModule):
    def __init__(self, grouper: CombinatorialGrouper, dom_mapper: DomainMapper, in_channels=3, num_cls=2, pretrain=None, *args: Any, **kwargs: Any) -> None:
        super().__init__(*args, **kwargs)


        # self.model = CNN(in_channels=in_channels, num_classes=num_cls)
        self.model = EarlyDANN(weights='scratch')

        self.criterion = torch.nn.CrossEntropyLoss()
        self.optimizer = torch.optim.SGD(params=self.parameters(), lr=1e-3, weight_decay=1e-2, momentum=0.9)
        self.metric = Accuracy(num_classes=2, task='multiclass')

        self.grouper: CombinatorialGrouper = grouper
        self.dom_mapper: DomainMapper = dom_mapper

    def training_step(self, batch, batch_idx):
        X, t, M = batch
        d = self.grouper.metadata_to_group(M.cpu())
        d = self.dom_mapper(d).cuda()

        y, z = self.model(X)
        
        loss_y = self.criterion(y, t)
        loss_d = self.criterion(z, d)

        self.log("loss_y", loss_y)
        self.log("loss_d", loss_d)

        return loss_y + loss_d
    
    def validation_step(self, batch, batch_idx):
        X, t, _ = batch

        y = self.model(X).argmax(dim=1)
        
        accuracy = self.metric(y, t)

        self.log('accuracy', accuracy)

        return accuracy
    
    def configure_optimizers(self) -> Any:
        return self.optimizer

In [5]:
transform = T.Compose([
    T.ToTensor()
])

dataset = get_dataset("camelyon17", root_dir="../../data/")
train_set = dataset.get_subset("train", transform=transform)
val_set = dataset.get_subset("id_val", transform=transform)
test_set = dataset.get_subset('test', transform=transform)

grouper = CombinatorialGrouper(dataset, ['hospital'])
dom_mapper = DomainMapper(train_set.metadata_array[:,0])

train_loader = get_train_loader("standard", train_set, grouper=grouper,uniform_over_groups=True, batch_size=32, num_workers=8)
val_loader = get_eval_loader('standard', val_set, batch_size=64, num_workers=8)
test_loader = get_eval_loader('standard', test_set, batch_size=64, num_workers=8)

In [6]:
trainer = L.Trainer(
    accelerator="auto", 
    max_epochs=10, 
    # val_check_interval=int(len(train_loader)/3)
)

model = SimpleCNN(grouper=grouper, dom_mapper=dom_mapper, in_channels=3, num_cls=2)

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs


In [7]:
trainer.fit(
    model,
    train_dataloaders=train_loader,
    # val_dataloaders=val_loader
)

/home/yasin/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/configuration_validator.py:71: PossibleUserWarning: You defined a `validation_step` but have no `val_dataloader`. Skipping val loop.
  rank_zero_warn(
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name      | Type               | Params
-------------------------------------------------
0 | model     | EarlyDANN          | 23.5 M
1 | criterion | CrossEntropyLoss   | 0     
2 | metric    | MulticlassAccuracy | 0     
-------------------------------------------------
23.5 M    Trainable params
0         Non-trainable params
23.5 M    Total params
94.095    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

/home/yasin/anaconda3/lib/python3.9/site-packages/pytorch_lightning/trainer/call.py:53: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
model_ = model.model.cuda()

: 

In [ ]:
BS = 32

: 

In [ ]:
grouper = CombinatorialGrouper(dataset, ['hospital'])

: 

In [ ]:
source_set = dataset.get_subset("train", transform=transform, frac=10_000/len(train_set))
target_set = dataset.get_subset("test", transform=transform, frac=10_000/len(test_set))

source_loader = get_eval_loader("standard", source_set, batch_size=BS, num_workers=8, grouper=grouper)
target_loader = get_eval_loader("standard", target_set, batch_size=BS, num_workers=8, grouper=grouper)

: 

In [ ]:
import umap
from sklearn.preprocessing import StandardScaler
import matplotlib.pyplot as plt

from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.metrics import classification_report, accuracy_score

from tqdm import tqdm

: 

In [ ]:
def plot_layer(ax, source: torch.Tensor, target: torch.Tensor, layer='x', acc: float=0.0):
    reducer = umap.UMAP(n_neighbors=5, n_epochs=1000, min_dist=0.1)
    all_embeddings = torch.cat([source[layer], target[layer]],dim=0)
    all_targets = torch.cat([source['t'], target['t']], dim=0)
    
    X_scaled = StandardScaler().fit_transform(all_embeddings)

    reducer = umap.UMAP(n_neighbors=5, min_dist=0.07)
    X_reduced = reducer.fit_transform(X_scaled)

    source_target = torch.cat([torch.zeros(10_000), torch.ones(10_000)])

    color_values = torch.cat([source_target[None,:], all_targets[None,:]], dim=0)

    color_map = {
        (0,0): "royalblue", # source, no_tumor
        (0,1): "mediumaquamarine", # source, tumor
        (1,0): "gold", # target, no_tumor
        (1,1): "goldenrod" # target, tumor
    }

    colors_master = [color_map[tuple(x.to(int).tolist())] for x in color_values.T]

    ax.scatter(
        X_reduced[:, 0],
        X_reduced[:, 1],
        c=colors_master,
        s=0.05
    )
    ax.set_xlabel(f"Layer: {layer}\n Accuracy: {acc}")

: 

In [ ]:
def predict_domain(source, target, layer='x'):
    X = torch.cat([source[layer], target[layer]], dim=0)
    t = torch.cat([source['d'], target['d']], dim=0)

    X_train, X_test, t_train, t_test = train_test_split(X, t, test_size=0.2, random_state=42)

    pipe = Pipeline([
        ('scaler', StandardScaler()), 
        ('lor', LogisticRegression(max_iter=400, C=0.01))
    ])

    pipe.fit(X_train, t_train)
    # print(classification_report(t_test, pipe.predict(X_test)))

    return accuracy_score(t_test, pipe.predict(X_test))

: 

In [ ]:
def compute_embeddings(model: nn.Module, source_loader, target_loader):
    num_samples_source = source_loader.dataset.__len__()
    num_samples_target = target_loader.dataset.__len__()

    source = {
        'z0': torch.empty((num_samples_source, 64)),
        'z1': torch.empty((num_samples_source, 256)),
        'z2': torch.empty((num_samples_source, 512)),
        'z3': torch.empty((num_samples_source, 1024)),
        'z4': torch.empty((num_samples_source, 2048)),
        'x': torch.empty((num_samples_source, 2048)),
        't': torch.empty((num_samples_source)),
        'd': torch.empty((num_samples_source)),
    }
    target = {
        'z0': torch.empty((num_samples_target, 64)),
        'z1': torch.empty((num_samples_target, 256)),
        'z2': torch.empty((num_samples_target, 512)),
        'z3': torch.empty((num_samples_target, 1024)),
        'z4': torch.empty((num_samples_target, 2048)),
        'x': torch.empty((num_samples_target, 2048)),
        't': torch.empty((num_samples_target)),
        'd': torch.empty((num_samples_target)),
    }

    with torch.no_grad():
        for i, (X, t, M) in enumerate(tqdm(source_loader)):
            bs = len(X)

            d = grouper.metadata_to_group(M)

            embs = model.embed(X.cuda())
            source['z0'][i*BS:i*BS+bs] = embs['z0'].cpu()
            source['z1'][i*BS:i*BS+bs] = embs['z1'].cpu()
            source['z2'][i*BS:i*BS+bs] = embs['z2'].cpu()
            source['z3'][i*BS:i*BS+bs] = embs['z3'].cpu()
            source['z4'][i*BS:i*BS+bs] = embs['z4'].cpu()
            source['x'][i*BS:i*BS+bs] = embs['x'].cpu()
            source['t'][i*BS:i*BS+bs] = t
            source['d'][i*BS:i*BS+bs] = d

        for i, (X, t, M) in enumerate(tqdm(target_loader)):
            bs = len(X)

            d = grouper.metadata_to_group(M)

            embs = model.embed(X.cuda())
            target['z0'][i*BS:i*BS+bs] = embs['z0'].cpu()
            target['z1'][i*BS:i*BS+bs] = embs['z1'].cpu()
            target['z2'][i*BS:i*BS+bs] = embs['z2'].cpu()
            target['z3'][i*BS:i*BS+bs] = embs['z3'].cpu()
            target['z4'][i*BS:i*BS+bs] = embs['z4'].cpu()
            target['x'][i*BS:i*BS+bs] = embs['x'].cpu()
            target['t'][i*BS:i*BS+bs] = t
            target['d'][i*BS:i*BS+bs] = d

    return source, target

: 

In [ ]:
def evaluate(model: nn.Module, source_loader, target_loader):
    source, target = compute_embeddings(model, source_loader, target_loader)

    # Compute domain accuracy
    acc_l0 = predict_domain(source, target, 'z0')
    acc_l1 = predict_domain(source, target, 'z1')
    acc_l2 = predict_domain(source, target, 'z2')
    acc_l3 = predict_domain(source, target, 'z3')
    acc_l4 = predict_domain(source, target, 'z4')

    # Plot embeddings
    fig, (ax0, ax1, ax2, ax3, ax4) = plt.subplots(1,5)

    fig.set_figheight(3)
    fig.set_figwidth(18)

    plot_layer(ax0, source, target, layer='z0', acc=acc_l0)
    plot_layer(ax1, source, target, layer='z1', acc=acc_l1)
    plot_layer(ax2, source, target, layer='z2', acc=acc_l2)
    plot_layer(ax3, source, target, layer='z3', acc=acc_l3)
    plot_layer(ax4, source, target, layer='z4', acc=acc_l4)

: 

In [ ]:
evaluate(model_, source_loader, target_loader)

: 